<a href="https://colab.research.google.com/github/bradleywjenks/CIVE_70019_70057/blob/main/notebooks/model_calibration_coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parameter estimation for hydraulic model calibration and fault detection


### CIVE 70019/70057
Department of Civil and Environmental Engineering, Imperial College London

### Preamble
You have been tasked with the evaluation and calibration of the hydraulic model of EXNING, a district metered area (DMA) in Anglian Water's (AW) water distribution network. To achieve this, you have been provided:
* a hydraulic model of the EXNING DMA, in use by Anglian Water in early 2019 (no record of recent calibration),
* pipe groups (based on material & age) and corresponding ranges of H-W coefficients,
* hourly loading conditions (demands, reservoir heads) and head measurements covering a period of 4 days in December 2019.

The objective of the coursework is to prepare a short calibration report for AW by completing the tasks below and answering the questions (max. 100-150 words per question) based on your results. Don't forget to include titles, labels and legends in your plots, and watch for significant figures in your reporting!

You have also been provided the following information about EXNING:
* EXNING is part of a larger system of cascading DMAs: EXNING is fed by the NEWSEV DMA and feeds into the BURWEL DMA.
* According to AW's existing records, EXNING contains mostly old, cast iron pipes.
* The "reservoir" head  and total demand of EXNING are derived from flow and pressure sensors at the DMA inlet (& outlet).
* Node elevations have been updated in the provided model following a GPS survey of sensor locations.

First, we must clone the GitHub repository and install dependencies (only run this once).

In [1]:
# run this cell once
import sys
import os

if 'google.colab' in sys.modules:
  !git clone https://github.com/bradleywjenks/CIVE_70019_70057.git
  !pip install wntr
  !pip install cvxpy
  !apt-get install libsuitesparse-dev && pip install scikit-sparse

Cloning into 'CIVE_70019_70057'...
remote: Enumerating objects: 726, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 726 (delta 66), reused 45 (delta 21), pack-reused 616
Receiving objects: 100% (726/726), 10.88 MiB | 9.85 MiB/s, done.
Resolving deltas: 100% (414/414), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 11.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libgraphblas-dev
  libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2 librbio2 libsliplu1 libspqr2
  libsuitesparseconfig5 libumfpack5
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libgraphblas-dev
  libgraphblas6 libklu1 libldl2 libmetis5 libmongoose2 librbio2 libsliplu1 l

In [2]:
# load packages
import numpy as np
from numpy import linalg as la
import networkx as nx
import pandas as pd
import wntr
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import copy
from datetime import datetime, timedelta
import cvxpy as cp
import warnings
warnings.filterwarnings('ignore')

# improve matplotlib image quality
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

### Load network properties and operational data

Load functions created in previous assignments.

In [3]:
# load functions from src folder
if 'google.colab' in sys.modules:
    sys.path.append('/content/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *
else:
    sys.path.append('/home/bradw/workspace/CIVE_70019_70057/src/')
    from general_functions import *
    from hydraulic_functions import *

Load network .inp file and operational data from the module repository's data directory.

In [4]:
if 'google.colab' in sys.modules:
    # if run in Google Colab
    data_dir = '/content/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/content/CIVE_70019_70057/data/networks/'
else:
    # replace with local directory
    data_dir = '/home/bradw/workspace/CIVE_70019_70057/data/parameter_estimation/'
    net_dir = '/home/bradw/workspace/CIVE_70019_70057/data/networks/'

net_name = 'exning.inp'
data_name = 'exning_coursework_dataset.npy'

# load operational data
data = np.load(os.path.join(data_dir, data_name), allow_pickle=True).item()
h_data = data['h_data']
sensor_idx = data['sensor_idx'] - 1 # matlab to python index offset
d_data = data['d']
h0_data = data['h0'].reshape(-1, 1).T

# load network properties
wdn = load_network_data(os.path.join(net_dir, net_name))
A12 = wdn.A12
A10 = wdn.A10
net_info = wdn.net_info
link_df = wdn.link_df
node_df = wdn.node_df
demand_df = wdn.demand_df

Plot sensor nodes in network. We provide a plotting function below.

In [ ]:
#### DO NOT CHANGE THIS ####
# network plotting function
def plot_network(wdn, sensor_idx, vals=None, highlight_valves=None):

    # unload data
    link_df = wdn.link_df
    node_df = wdn.node_df
    net_info = wdn.net_info
    h0_df = wdn.h0_df

    # draw network
    uG = nx.from_pandas_edgelist(link_df, source='node_out', target='node_in')
    pos = {row['node_ID']: (row['xcoord'], row['ycoord']) for _, row in node_df.iterrows()}
    nx.draw(uG, pos, node_size=20, node_shape='o')

    # draw reservoir
    nx.draw_networkx_nodes(uG, pos, nodelist=net_info['reservoir_names'], node_size=100, node_shape='s', node_color='black')

    # draw sensor nodes
    sensor_names = [net_info['junction_names'][i] for i in sensor_idx]
    nx.draw_networkx_nodes(uG, pos, sensor_names, node_size=100, node_shape='o', node_color='red', edgecolors='white')

    # reservoir labels
    reservoir_labels = {node: 'Reservoir' for node in net_info['reservoir_names']}
    labels_res = nx.draw_networkx_labels(uG, pos, reservoir_labels, font_size=12, verticalalignment='top')
    for _, label in labels_res.items():
        label.set_y(label.get_position()[1] - 1500)

    # sensor labels
    sensor_labels = {node: str(idx+1) for (idx, node) in enumerate(sensor_names)}
    labels_sen = nx.draw_networkx_labels(uG, pos, sensor_labels, font_size=12, verticalalignment='bottom')
    for _, label in labels_sen.items():
        label.set_y(label.get_position()[1] + 1000)

    # plot sensor vals
    if vals is not None:

        cmap = cm.get_cmap('RdYlGn_r')

        # plot residuals
        nx.draw_networkx_nodes(uG, pos, nodelist=sensor_names, node_size=100, node_shape='o', node_color=vals, cmap=cmap, edgecolors='white')

        # create color bar
        sm = plt.cm.ScalarMappable(cmap=cmap)
        sm.set_array(vals)
        colorbar = plt.colorbar(sm)
        colorbar.set_label('Mean pressure residual [m]', fontsize=12)

    # highlight link
    if highlight_valves is not None:
        nx.draw_networkx_nodes(uG, pos, highlight_valves, node_size=200, node_shape='d', node_color='limegreen', edgecolors='white')

In [ ]:
plot_network(...)

### Preliminary model evaluation

Simulate initial (uncalibrated) network hydraulics over 4 days. We first define a function to solve network hydraulics using the `wntr` package, which we used previously in the hyraulic modelling notebook. The following tasks are performed in this function:
- Load network properties
- Modify simulation time to match operational data
- Assign h0 data at model reservoir
- Scale and apply new demand pattern from inflow data
- Option to modify pipe roughness (or HW) coefficients

<font color="blue">NB: this is done for you below. Provided the correct inputs, you do not need to replicate this function.

In [ ]:
#### DO NOT CHANGE THIS ####
# hydraulic solver function
def hydraulic_solver(inp_file, d_data, h0_data, C=None, demand=False):

    # load network from wntr
    inp_file = os.path.join(net_dir, net_name)
    wn = wntr.network.WaterNetworkModel(inp_file)

    # get network properties
    reservoir_names = wn.reservoir_name_list
    junction_names = wn.junction_name_list
    link_names = wn.link_name_list

    # modify simulation time and hydraulic time step
    nt = h0_data.shape[1]
    wn.options.time.duration = (nt - 1) * 3600
    wn.options.time.hydraulic_timestep = 3600
    wn.options.time.pattern_timestep = 3600
    wn.options.time.report_timestep = 3600

    # assign reservoir data
    for (i, name) in enumerate(reservoir_names):
        wn.add_pattern(f'h0_{i}', h0_data[i])
        reservoir = wn.get_node(name)
        reservoir.head_timeseries.base_value = 1
        reservoir.head_timeseries.pattern_name = wn.get_pattern(f'h0_{i}')

    # replace demand data
    for idx, name in enumerate(junction_names):

        if any(val != 0 for val in d_data[idx, :]):
            node = wn.get_node(name)
            d_pat = d_data[idx, :]
            wn.add_pattern('d_'+name, d_pat)

            for (i, num) in enumerate(node.demand_timeseries_list):
                if i == 0:
                    node.demand_timeseries_list[i].base_value = 1
                    node.demand_timeseries_list[i].pattern_name = 'd_'+name
                else:
                    node.demand_timeseries_list[i].base_value = None
                    node.demand_timeseries_list[i].pattern_name = None

    # assign roughness (or HW) coefficients
    if C is not None:
        for name, link in wn.links():

            # check if the link is a pipe
            if isinstance(link, wntr.network.Pipe):
                link.roughness = C[link_names.index(name)]

            # check if link is a valve
            elif isinstance(link, wntr.network.Valve):
                link.minor_loss = C[link_names.index(name)]
                link.initial_setting = C[link_names.index(name)]

    # run simulation and get results
    sim = wntr.sim.EpanetSimulator(wn)
    results = sim.run_sim()

    q_sim = results.link['flowrate'].T
    h_sim = results.node['head'].T
    h_sim = h_sim[~h_sim.index.isin(reservoir_names)] # delete reservoir nodes
    d = results.node['demand'].T
    d = d[~d.index.isin(reservoir_names)] # delete reservoir nodes


    if demand == True:
        return d.to_numpy()
    else:
        return q_sim.to_numpy(), h_sim.to_numpy()

Run simulation with initial $C_0$ values.

In [ ]:
C_0 = link_df['C'].to_numpy()
_, h_0 = hydraulic_solver(...)

Compare simulated heads at sensor nodes (node indices in `sensor_idx`) with the simulated heads over the 4-day period. Visualise the results with, e.g. a boxplot of pressure residuals on the test dataset.

In [ ]:
# compute pressure residuals
residuals_0 = ...

**<u>Question 1:</u>** Is the current hydraulic model of EXNING accurate according to hydraulic model calibration guidelines? Comment on the results of the preliminary model evaluation and, in particular, on
* the **sign** (i.e. are heads over or underestimated in the hydraulic simulation?) of pressure/head residuals,
* the **temporal** distribution (i.e. are the residuals time/flow dependent?) of pressure/head residuals,
* and the **spatial** distribution (i.e. are all sensors affected?) of the pressure/head residuals.

Given the information you were provided about the EXNING system and the results of the preliminary model evaluation, identify the most likely sources of model errors.

<font color="red">Enter response here...

### Part 1: parameter estimation without pipe grouping + valves fixed

Based on the outcome of the initial comparison between measured and simulated heads at sensor nodes, you are tasked with the calibration of the hydraulic model of EXNING. Given the information provided by Anglian Water about the EXNING system, the first step is to adjust the pipe roughness (H-W) coefficients. In order to limit the number of parameters in question, and regardless of your answer to question 1, you will first assume (in Parts 1 and 2 of the coursework) that the status of valves is known and minor/local loss coefficients are fixed to C=0.2 (fully open).

First, you will investigate the effect of pipe grouping on the calibration of H-W coefficients. In Part 1, you will solve the model calibration problem without pipe grouping. Complete the code below to calibrate the network model using the head measurements loaded previously.

#### Separate the data into train and test datasets
We suggest you use the first day worth of data (loading conditions + $h_0$ measurements) as a train dataset, and the remaining three (3) days as a test dataset.

In [ ]:
# tain data
nt_train = 24
data_train = {
    'd': ...,
    'h0': ...,
    'h_data': ...
}

# test data
data_test = {
    'd': ...,
    'h0': ...,
    'h_data': ...
}

**<u>Question 2:</u>** Briefly comment on the definition of the train dataset. What impact may it have on the predictive ability of your model? (i.e. what range of conditions will you confidently be able to use your model for?)

<font color="red">Enter response here...

#### Definition of the loss function
The calibration of H-W coefficients is a model fitting/parameter estimation problem. In particular, for the hydraulic model calibration problem, the loss function is defined as the mean squared error (MSE) between simulated and measured heads at sensor locations:

In [ ]:
def loss_fun(h, h_data):
    return ( 1/len(h_data.flatten()) ) * np.sum( ( ... - ... )**2 )

Compute MSE for `C_0` values and using the training data.

In [ ]:
h_0 = ...
mse_0 = loss_fun(...)
mse_0

**<u>Question 3:</u>**  Justify the choice/definition of `loss_fun`.

<font color="red">Enter response here...

#### Solve the parameter estimation problem using the train dataset

The following function is needed for the sequential convex programming (SCP) method used in this coursework. <font color="blue">As with the `hydraulic_solver` function, we provide the following code in `linear_approx_calibration` for you to use throughout this notebook.

In [ ]:
#### DO NOT CHANGE THIS ####
# compute matrices for the head loss linear approximation of the HW headloss
def linear_approx_calibration(wdn, q, C):
    # unload data
    A12 = wdn.A12
    A10 = wdn.A10
    net_info = wdn.net_info
    link_df = wdn.link_df

    K = np.zeros((net_info['np'], 1))
    n_exp = link_df['n_exp'].astype(float).to_numpy().reshape(-1, 1)
    b1_k = copy.copy(K)
    b2_k = copy.copy(K)

    for idx, row in link_df.iterrows():
        if row['link_type'] == 'pipe':
            K[idx] = 10.67 * row['length'] * (C[idx] ** -row['n_exp']) * (row['diameter'] ** -4.8704)
            b1_k[idx] = copy.copy(K[idx])
            b2_k[idx] = (-n_exp[idx] * K[idx]) / C[idx]

        elif row['link_type'] == 'valve':
            K[idx] = (8 / (np.pi ** 2 * 9.81)) * (row['diameter'] ** -4) * C[idx]
            b1_k[idx] = -n_exp[idx] * copy.copy(K[idx])
            b2_k[idx] = copy.copy(K[idx]) / C[idx]

    a11_k = np.tile(K, q.shape[1]) * np.abs(q) ** (n_exp - 1)
    b1_k = np.tile(b1_k, q.shape[1]) * np.abs(q) ** (n_exp - 1)
    b2_k = np.tile(b2_k, q.shape[1]) * np.abs(q) ** (n_exp - 1) * q

    return a11_k, b1_k, b2_k

Solve the model calibration problem without pipe grouping using the `cvxpy` modelling interface. (Use the code provided previously.)

In [ ]:
# unload network training data
n_exp = link_df['n_exp']
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define SCP problem parameters
Ki = np.inf
iter_max = 50
delta_k = 20
C_up_pipe = 200
C_lo_pipe = 1e-4

# initialise values
theta_k = ...
q_k, h_k = hydraulic_solver(...)
a11_k, b1_k, b2_k = linear_approx_calibration(...)
objval_k = loss_fun(...)

### main scp code ###
for k in range(iter_max):

    # insert code here....

**<u>Question 4:</u>** Describe the nature/role of the different outputs of the SCP algorithm printed above (objvalk, Ki, Deltak) and explain their trends.

<font color="red">Enter response here...

Store the solution (i.e. new coefficients `theta_k`) as $C_1$.

In [ ]:
C_1 = ...

#### Evaluate test model error
Compute the MSE and plot the pressure residuals corresponding to the test dataset.

In [ ]:
# compute hydraulics
_, h_test = hydraulic_solver(...)

# compute test mse
mse_test_1 = loss_fun(...)
mse_test_1

In [ ]:
# compute pressure residuals
residuals_1 = ...

**<u>Question 5:</u>** Comment on the improvement in model accuracy (before vs. after calibration).

<font color="red">Enter response here...

#### Discuss the values of the calibrated H-W coefficients $C_1$
Visualise the values of newly calibrated coefficients $C_1$ compared to original model coefficients $C_0$.

In [ ]:
# generate C_1 v. C_0 value scatter plot (pipes only)

**<u>Question 6:</u>** Comment on the values of parameter estimates in $C_1$ and explain the results of the calibration without pipe grouping. Can the new model be considered calibrated?

<font color="red">Enter response here...

### Part 2: parameter estimation with pipe grouping + valves fixed
In order to reduce the underdeterminedness of the model calibration problem (and improve the quality of estimated H-W coefficients), pipes can be grouped based on their material and age. In particular, all pipes of a group are assumed to share the same H-W coefficient. This also allows tighter bounds on the grouped coefficient estimates (stored in $\Theta_{\text{lb}}$ and $\Theta_{\text{ub}}$, for lower and upper bounds, respectively) in the formulation of the parameter estimation problem. Here, you will still assume the status of valves is known with minor/local loss coefficients fixed to C=0.2. Use the code provided in Week 5 and modify as necessary below to calibrate the hydraulic model with pipe grouping, using the same train data as before. (Note that Part 2 might take a little longer to run than Part 1.)

Load pipe grouping information.

In [ ]:
link_groups = data['G'][0] - 1 # matlab to python index offset
ng = link_groups.shape[0]
thetaG_lo = data['thetaG_min'][:, 0]
thetaG_up = data['thetaG_max'][:, 0]

#### Solve the parameter estimation problem using the train dataset
Solve the model calibration problem with pipe grouping. (Use the code provided previously and modify as necessary.)

In [ ]:
# unload network training data
n_exp = link_df['n_exp']
d = data_train['d']
h_data = data_train['h_data']
h0 = data_train['h0'].reshape(-1, 1).T

# define problem parameters
Ki = np.inf
iter_max = 50
delta_k = 20
C_up_pipe = 200
C_lo_pipe = 0

# initialise values
theta_k = ...
q_k, h_k = hydraulic_solver(...)
a11_k, b1_k, b2_k = linear_approx_calibration(...)
objval_k = loss_fun(...)

### main scp code ###
for k in range(iter_max):

    # insert code here...

Store the solution (i.e. new coefficients `theta_k`) as $C_2$.

In [ ]:
C_2 = ...

# compute train mse
mse_train_2 = loss_fun(...)
mse_train_2

#### Evaluate test model error
Compute the MSE and plot the pressure residuals corresponding to the test dataset.

In [ ]:
# compute hydraulics
_, h_test = hydraulic_solver(...)

# compute test mse
mse_test_2 = loss_fun(...)
mse_test_2

In [ ]:
# compute pressure residuals
residuals_2 = ...

**<u>Question 7:</u>** Comment on the improvement in model accuracy after calibration with pipe grouping.

<font color="red">Enter response here...

#### Discuss the values of the calibrated H-W coefficients $C_2$
Visualise the values of newly calibrated coefficients $C_2$ compared to model coefficients $C_1$ and $C_0$.

In [ ]:
# generate C_2 value scatter plot (pipes only)

**<u>Question 8:</u>** Comment on the values of the H-W coefficient estimates $C_2$ and justify the results of the calibration with pipe grouping.

<font color="red">Enter response here...

**<u>Question 9:</u>** Can the newly calibrated model (with H-W coefficients in $C_2$) be considered calibrated? Considering the results of Part 2, as well as your answer to question 1, provide an interpretation of the remaining pressure/head residuals.

In [ ]:
# spatial residuals plot (hint: you can use `plot_network` function`)

<font color="red">Enter response here...

### Part 3: parameter estimation with pipe grouping + valve coefficients free

Following initial reports concerning discrepancies in the EXNING model, AW were able to confirm that:
* flow and head sensors had been calibrated before collection of load/field data corresponding to the train and test datasets,
* allocation of demands in the train and test dataset is representative of normal network conditions and based on recent demand monitoring campaigns (incl. of large industrial users), and
* boundary valves (isolating EXNING from adjacent DMAs) are closed.

As a result, the remaining deviations between measured and simulated pressures must result from unknown/unreported changes in system connectivity. Errors associated with unknown valve status (e.g. closed valves) can be identified by solving a similar parameter estimation problem where valve coefficients, in addition to H-W coefficients, are free to vary.

In Part 3, you are encouraged to propose and investigate an approach to calibrate the hydraulic model with unknown valve coefficients. You may follow the suggested steps below, or come up with your own. Marks will be allocated based on the justification of the adopted approach (show your thinking!) and discussion of the obtained results in question 10, whether they lead to a firm conclusion about the existence/location of unknowingly closed valves or not.

Suggested approach:
1. Try modifying the CVXPY implementation in Part 2 to allow valve coefficients to vary. Solve the resulting parameter estimation problem and interpret the optimal value of $C_3$.
2. Next, try further modifying your CVXPY implementation to account for the expected sparsity of large valve coefficients (hint: you might want to consider regularising your problem formulation). Solve the resulting problem and comment on the optimal value of $C_3$.
3. Finally, compare the results of Part 3 with your answer to question 1 and preliminary model evaluation. Explain whether they contradict or corroborate your previous conclusions about the most likely sources of error in the EXNING model by answering question 10 below.

In [ ]:
# insert Part 3 code here...

**<u>Question 10:</u>** Summarise your findings and provide recommendations to AW to validate your proposed hydraulic model update. Provide plots to support your recommendations.

<font color="red">Enter response here...